In [143]:
import tensorflow as tf
from tensorflow import keras, sparse 
from keras import layers, applications
# from keras.preprocessing import image
from keras.applications.efficientnet import EfficientNetB0, EfficientNetB5, EfficientNetB7
from keras.applications.resnet_v2 import ResNet152V2
from keras.applications.densenet import DenseNet121
# from keras.applications import EfficientNetB0
from keras.applications.efficientnet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator


import os 
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential

# Data preprocessing 
from keras.preprocessing.image import load_img, img_to_array
 

In [144]:
def getDataset(dataFolder, subset, imageSize = (224, 224), batchSize = 32):
    train_ds = keras.utils.image_dataset_from_directory(
      dataFolder,
      seed=123,
      image_size=imageSize,
      batch_size=batchSize)
    return train_ds


def configurePerformance(train_ds, val_ds): 
    AUTOTUNE = tf.data.AUTOTUNE
    train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
    return train_ds, val_ds

# def buildModel(dropoutRate, numClasses, inpShape = (224, 224, 3)):
import pickle

def dumpModel(modelName, phase): 
    # Save the trained model as a pickle string.
    modelName = "model_" + modelName + "_ " + phase + ".pkl"
    pickle.dump(model, open(modelName, 'wb'))




In [145]:
EPOCHS = 5
modelName = "efficientNetB0"
inpShape =  (224, 224, 3)
trainingFolder = 'data/x5_RGB/train/RGB/'
testingFolder = 'data/x5_RGB/test_with_labels/RGB/'
train_ds = getDataset(trainingFolder, "training")
val_ds =  getDataset(testingFolder, "validation")


Found 6000 files belonging to 7 classes.
Found 1500 files belonging to 7 classes.


In [146]:
dropoutRate = 0.2
numClasses = 7 
inp = layers.Input(shape=inpShape)
baseModel = EfficientNetB0(weights="imagenet",
                   include_top = False) 

baseModel.trainable = False 
x = baseModel(inp, training=False)
x =  layers.GlobalAveragePooling2D(name="avg_pool")(x)
x = layers.Dropout(dropoutRate, noise_shape=None, seed=None)(x)
out = layers.Dense(numClasses,activation="softmax", name = "pred")(x)
model = keras.Model(inp, out, name="FeatureExtraction-B0")
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
          loss=tf.keras.losses.SparseCategoricalCrossentropy(),
          metrics=['accuracy'])

In [147]:
# Feature extraction without the top layers 
hist_results = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=EPOCHS
)

dumpModel(modelName, "phase1")


Epoch 1/5


2022-03-06 18:37:11.551205: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


188/188 [==============================] - ETA: 0s - loss: 0.3980 - accuracy: 0.8618

2022-03-06 18:37:55.089202: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


188/188 [==============================] - 59s 294ms/step - loss: 0.3980 - accuracy: 0.8618 - val_loss: 0.9846 - val_accuracy: 0.7013
Epoch 2/5
188/188 [==============================] - 56s 295ms/step - loss: 0.2322 - accuracy: 0.9192 - val_loss: 1.0262 - val_accuracy: 0.6960
Epoch 3/5
188/188 [==============================] - 55s 290ms/step - loss: 0.1858 - accuracy: 0.9342 - val_loss: 1.3487 - val_accuracy: 0.6740
Epoch 4/5
188/188 [==============================] - 53s 278ms/step - loss: 0.1781 - accuracy: 0.9350 - val_loss: 1.1395 - val_accuracy: 0.7180
Epoch 5/5
188/188 [==============================] - 53s 280ms/step - loss: 0.1805 - accuracy: 0.9362 - val_loss: 1.2097 - val_accuracy: 0.7233
INFO:tensorflow:Assets written to: ram://6aefa8d1-5dda-416c-8dcc-681d43097ad8/assets


In [149]:
# Fine tuning the Feature Extraction Model 
baseModel.trainable = True
for layer in model.layers[1].layers:
    if isinstance(layer, layers.BatchNormalization):
        layer.trainable = False
        
model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics = ["accuracy"])



In [150]:
# Train it again 
hist_results_tuned = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=EPOCHS
)

dumpModel(modelName, "phase2")


Epoch 1/5


2022-03-06 18:42:56.020300: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


188/188 [==============================] - ETA: 0s - loss: 0.1391 - accuracy: 0.9530

2022-03-06 18:46:02.375290: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


188/188 [==============================] - 202s 1s/step - loss: 0.1391 - accuracy: 0.9530 - val_loss: 1.1900 - val_accuracy: 0.7293
Epoch 2/5
188/188 [==============================] - 185s 981ms/step - loss: 0.0498 - accuracy: 0.9810 - val_loss: 1.3069 - val_accuracy: 0.7120
Epoch 3/5
188/188 [==============================] - 185s 985ms/step - loss: 0.0207 - accuracy: 0.9927 - val_loss: 1.1580 - val_accuracy: 0.7733
Epoch 4/5
188/188 [==============================] - 185s 984ms/step - loss: 0.0135 - accuracy: 0.9962 - val_loss: 1.1539 - val_accuracy: 0.7800
Epoch 5/5
188/188 [==============================] - 200s 1s/step - loss: 0.0083 - accuracy: 0.9973 - val_loss: 1.6415 - val_accuracy: 0.7100
INFO:tensorflow:Assets written to: ram://a506b280-e7e8-4272-8bba-d3da429fb18e/assets


INFO:tensorflow:Assets written to: ram://cb986ee5-de15-4079-9011-3659a85c3883/assets


In [126]:
preds = model.predict(val_ds, verbose = 1)
model.evaluate(val_ds)

2022-03-05 17:47:34.977248: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


47/47 [==============================] - 14s 246ms/step


In [130]:
# def getImageRepresentation():
#     import random
#     pred_labels = tf.argmax(preds, axis=1)
#     test_labels = np.concatenate([y for x, y in val_ds], axis=0)
#     test_image_batches = []
#     for images, labels in val_ds.take(-1):
#         test_image_batches.append(images.numpy())

#     test_images = [item for sublist in test_image_batches for item in sublist]
#     plt.figure(figsize = (20,20))
#     for i in range(9):
#         random_int_index = random.choice(range(len(test_images)))
#         plt.subplot(3,3,i+1)
#         plt.imshow(test_images[random_int_index]/255.)
#         if test_labels[random_int_index] == pred_labels[random_int_index]:
#             color = "g"
#         else:
#             color = "r"
#         plt.title("True Label: " + class_names[test_labels[random_int_index]] + " || " + "Predicted Label: " +
#                   class_names[pred_labels[random_int_index]] + "\n" + 
#                   str(np.asarray(tf.reduce_max(preds, axis = 1))[random_int_index]), c=color)
#         plt.axis(False);

47/47 [==============================] - 12s 228ms/step - loss: 0.9283 - accuracy: 0.7053


[0.9282761216163635, 0.7053333520889282]